In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

In [ ]:
instruments = ['hc','rb','i','j','jm','au','ag','v','ru','l','pp','bu','TA','FG','MA',
               'y','p','m','a','c','cs','jd','RM','CF','SR','OI']
start_date = 20180101
table = "AdjustedFuturesDaily"
engine = create_engine("sqlite:///../data/FuturesMarketData.db")
in_binds = ", ".join([f":sym{i}" for i in range(len(instruments))])
sql = text(f"""
        SELECT *, (ClosePrice * factor_multiply) as adjclose
        FROM {table}
        WHERE TradingDay >= {start_date}
            AND Instrument IN ({in_binds})
            AND method = 'OpenInterest'
    """)

params = {"start": start_date} | {f"sym{i}": s for i, s in enumerate(instruments)}

with engine.begin() as conn:
    df = pd.read_sql(sql, conn, params=params)

df = df.pivot(index="TradingDay", columns="Instrument")
df["adjclose"]


In [ ]:
px = df["adjclose"].sort_index()
px = px.where(px > -1)

lookback = 15
skip = 1

# L = 15
# s = 1
# logp = np.log(px)

# mode = "simple"
# signal = logp.shift(s) - logp.shift(L - s)
# signal

# mode = "linear"
log_return = np.log(px / px.shift(1))

weights = np.arange(lookback, 0, -1, dtype=float)
weights /= weights.sum()

signal = (
    log_return
    .rolling(lookback)
    .apply(lambda x: np.dot(x, weights), raw=True)
)
signal

In [ ]:
window = 15
trade_percent = 0.2
gross_target = 1.0
hold_period = 2
data = df["adjclose"]

low = data.quantile(trade_percent, axis=1)
high = data.quantile(1 - trade_percent, axis=1)

longs = (data.ge(high, axis=0)).astype(float)
shorts = (data.le(low, axis=0)).astype(float) * -1

weights = longs + shorts
weights = weights.sub(weights.mean(axis=1), axis=0)

gross = weights.abs().sum(axis=1)
scale = gross_target / gross.replace(0, np.nan)
position = weights.mul(scale, axis=0).fillna(0)
position

In [ ]:
price_diff = data.diff(periods=1)
pos_shift = position.shift(1).fillna(0)
pnl = pos_shift * price_diff

# Portfolio PnL
pnl_ptf = pnl.sum(axis=1)

# Turnover (sum of absolute position changes)
turnover = (position - pos_shift).abs().sum(axis=1)